In [1]:
import numpy as np
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score
import scipy.stats as st
import sys
import os
sys.path.append('../quantnn-keras/layers/')

In [26]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from keras.optimizers import SGD, Adam
from keras.losses import squared_hinge
from keras.layers import Flatten, Dense, Activation, BatchNormalization, Conv1D

from keras.models import Sequential, Model
import numpy as np

from binary_layers import BinaryConv2D, BinaryConv1D
from binary_ops import binary_tanh as binary_tanh_op
from keras.regularizers import l2

In [48]:
data_input_file = 'data/LOSO/MHEALTH.npz'

tmp = np.load(data_input_file)
X = tmp['X']
y = tmp['y']
folds = tmp['folds']

n_class = y.shape[1]

avg_acc = []
avg_recall = []
avg_f1 = []
y = np.argmax(y, axis=1)

print('ConvNet Template 2017 {}'.format(data_input_file))

for i in range(0, len(folds)):
    train_idx = folds[i][0]
    test_idx = folds[i][1]

    X_train, y_train = X[train_idx], y[train_idx]
    X_test, y_test = X[test_idx], y[test_idx]
    X_train = X_train.squeeze()
    X_test = X_test.squeeze()
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    
    # Create Model
    model = Sequential()

    _momentum = 0.1
    _epsilon = 0.001
    
    model.add(
        BinaryConv1D(
            kernel_size=30,
            filters=64,
            padding='same',
            input_shape=(250,23),
            kernel_lr_multiplier=10,use_bias=False))
    model.add(BatchNormalization(momentum=_momentum, epsilon=_epsilon))
    model.add(Activation(binary_tanh_op))


    model.add(
        BinaryConv1D(
            kernel_size=30, filters=128, padding='same',
            kernel_lr_multiplier=10,use_bias=False))
    model.add(BatchNormalization(momentum=_momentum, epsilon=_epsilon))
    model.add(Activation(binary_tanh_op))


    model.add(
        BinaryConv1D(
            kernel_size=30, filters=256, padding='same',
            kernel_lr_multiplier=10,use_bias=False))
    model.add(BatchNormalization(momentum=_momentum, epsilon=_epsilon))
    model.add(Activation(binary_tanh_op))

    model.add(Flatten())
    model.add(Dense(12, use_bias=False))
    model.add(BatchNormalization(momentum=_momentum, epsilon=_epsilon))

    # Train
    adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile(loss=squared_hinge, optimizer=adam, metrics=['accuracy'])
    #model.summary()

    model.fit(
        X_train, y_train, batch_size=50, epochs=100, validation_data=(X_test,y_test), verbose=0 )
    print (model.evaluate(X_test,y_test))

ConvNet Template 2017 data/LOSO/MHEALTH.npz
255/255 [==============================] - 0s 413us/step
[0.9179191189653734, 0.7960784323075238]
263/263 [==============================] - 0s 433us/step
[0.921292200061305, 0.5779467682874702]
251/251 [==============================] - 0s 461us/step
[0.9293723239366751, 0.44223107522227373]
266/266 [==============================] - 0s 332us/step
[0.9186151646133652, 0.6315789478165763]
265/265 [==============================] - 0s 460us/step
[0.9184935306603054, 0.6867924528301886]
264/264 [==============================] - 0s 472us/step
[0.9182958512595205, 0.6856060606060606]
253/253 [==============================] - 0s 428us/step
[0.9245998352883833, 0.36363636304738495]
239/239 [==============================] - 0s 371us/step
[0.9211378686098873, 0.6861924688062907]
251/251 [==============================] - 0s 437us/step
[0.9332977623578562, 0.609561756075141]
248/248 [==============================] - 0s 417us/step
[0.92052473752729

In [4]:
print (X_train.shape, X_test.shape)
print (y_train.shape, y_test.shape)

(2307, 1, 250, 23) (248, 1, 250, 23)
(2307,) (248,)


In [11]:
X_train = X_train.squeeze()
X_test = X_test.squeeze()
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [12]:
print (X_train.shape, X_test.shape)
print (y_train.shape, y_test.shape)

(2307, 250, 23) (248, 250, 23)
(2307, 12) (248, 12)


In [13]:
print (folds.shape, len(folds))

(10, 2) 10


In [43]:
# Create Model
model = Sequential()

model.add(
    BinaryConv1D(
        kernel_size=30,
        filters=64,
        padding='same',
        input_shape=(250,23),
        kernel_lr_multiplier=10,use_bias=False))
model.add(BatchNormalization(momentum=0.1, epsilon=0.0001))
model.add(Activation(binary_tanh_op))


model.add(
    BinaryConv1D(
        kernel_size=30, filters=128, padding='same',
        kernel_lr_multiplier=10,use_bias=False))
model.add(BatchNormalization(momentum=0.1, epsilon=0.0001))
model.add(Activation(binary_tanh_op))


model.add(
    BinaryConv1D(
        kernel_size=30, filters=256, padding='same',
        kernel_lr_multiplier=10,use_bias=False))
model.add(BatchNormalization(momentum=0.1, epsilon=0.0001))
model.add(Activation(binary_tanh_op))

model.add(Flatten())
model.add(Dense(12, use_bias=False))
model.add(BatchNormalization(momentum=0.1, epsilon=0.0001))

# Train
adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss=squared_hinge, optimizer=adam, metrics=['accuracy'])
model.summary()

model.fit(
    X_train, y_train, batch_size=50, epochs=100, validation_data=(X_test,y_test), verbose=0 )
model.evaluate(X_test,y_test)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
binary_conv1d_37 (BinaryConv (None, 250, 64)           44160     
_________________________________________________________________
batch_normalization_61 (Batc (None, 250, 64)           256       
_________________________________________________________________
activation_45 (Activation)   (None, 250, 64)           0         
_________________________________________________________________
binary_conv1d_38 (BinaryConv (None, 250, 128)          245760    
_________________________________________________________________
batch_normalization_62 (Batc (None, 250, 128)          512       
_________________________________________________________________
activation_46 (Activation)   (None, 250, 128)          0         
_________________________________________________________________
binary_conv1d_39 (BinaryConv (None, 250, 256)          983040    
__________

[0.9192171500575158, 0.6209677428968491]

In [30]:
# Create Model
model = Sequential()

model.add(
    Conv1D(
        kernel_size=10,
        filters=64,
        padding='same',
        input_shape=(250,23)))
model.add(BatchNormalization(momentum=0.1, epsilon=0.0001))
model.add(Activation('relu'))
model.add(Conv1D(
        kernel_size=10, filters=64, padding='same'))
model.add(BatchNormalization(momentum=0.1, epsilon=0.0001))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(12, use_bias=False))
model.add(BatchNormalization(momentum=0.1, epsilon=0.0001))

# Train
adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss=squared_hinge, optimizer=adam, metrics=['accuracy'])

model.summary()
model.fit(
    X_train, y_train, batch_size=50, epochs=100, validation_data=(X_test,y_test))
model.evaluate(X_test,y_test)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_8 (Conv1D)            (None, 250, 64)           14784     
_________________________________________________________________
batch_normalization_26 (Batc (None, 250, 64)           256       
_________________________________________________________________
activation_20 (Activation)   (None, 250, 64)           0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 250, 64)           41024     
_________________________________________________________________
batch_normalization_27 (Batc (None, 250, 64)           256       
_________________________________________________________________
activation_21 (Activation)   (None, 250, 64)           0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 16000)             0         
__________

2307/2307 [==============================] - 0s 152us/step - loss: 0.9167 - acc: 0.8479 - val_loss: 0.9177 - val_acc: 0.8024
Epoch 50/100
2307/2307 [==============================] - 0s 145us/step - loss: 0.9167 - acc: 0.8457 - val_loss: 0.9180 - val_acc: 0.6089
Epoch 51/100
2307/2307 [==============================] - 0s 158us/step - loss: 0.9167 - acc: 0.8539 - val_loss: 0.9177 - val_acc: 0.7702
Epoch 52/100
2307/2307 [==============================] - 0s 145us/step - loss: 0.9167 - acc: 0.8422 - val_loss: 0.9195 - val_acc: 0.6774
Epoch 53/100
2307/2307 [==============================] - 0s 152us/step - loss: 0.9167 - acc: 0.8305 - val_loss: 0.9173 - val_acc: 0.6250
Epoch 54/100
2307/2307 [==============================] - 0s 155us/step - loss: 0.9167 - acc: 0.8405 - val_loss: 0.9175 - val_acc: 0.7258
Epoch 55/100
2307/2307 [==============================] - 0s 151us/step - loss: 0.9167 - acc: 0.8483 - val_loss: 0.9176 - val_acc: 0.8629
Epoch 56/100
2307/2307 [=======================

248/248 [==============================] - 0s 191us/step


[0.9168156596922106, 0.6653225806451613]